# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Шаги исследования

1. Загрузка данных
2. Умножение матриц
3. Алгоритм преобразования
4. Проверка алгоритма
5. Общий вывод

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from numpy.linalg import inv


data = pd.read_csv(r"C:\Users\stasf\Desktop\insurance.csv")

In [2]:
# Сделаем функцию для описания данных
def information(df):
    print("Количество строк:", df.shape[0])
    print("Количество столбцов:", df.shape[1])
    print("Количество дубликатов:", df.duplicated().sum())
    df.info()
    display(df.describe())
    display(df.head(5))

In [3]:
information(data)

Количество строк: 5000
Количество столбцов: 5
Количество дубликатов: 153
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Данные загружены и изучены. Пропуски отсутствуют, количество дубликатов - 153. Скорее всего, это просто клиенты с идентичными признаками.

In [4]:
# Разделим данные на фичи и таргеты
data_features = data.drop('Страховые выплаты', axis=1)
data_target = data['Страховые выплаты']

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении матрицы признаков на обратимую матрицу качество линейной регрессии **не изменится**.

**Обоснование:** Домножим матрицу признаков $X$ на квадратную матрицу $P$ (размером 4 на 4), при условии что она обратима. Тогда формула предсказания будет выглядеть так: 
$$ a' = XPw' $$ 
где формула обучения будет представлена как:
$$ w' = ((XP)^{T}XP)^{-1}(XP)^{T}y $$ 

Для доказтельства будем использовать свойства операций над матрицами.
Поделим доказательство на несколько шагов:

**1**. $(XP)^{T} = P^{T}X^{T}$. 
Получим: 

$$ w' = (P^{T}X^{T}XP)^{-1}P^{T}X^{T}y $$ 

**2**. Так как обратимыми являются только квадратные матрицы, то проведем такое преобразование: $P^{T}X^{T}XP = (P^{T}X^{T}X)P$

$$ w' = ((P^{T}X^{T}X)P)^{-1}P^{T}X^{T}y $$

**3**. В результате матричного умножения в скобке $(P^{T}X^{T}X)$ получится квадртаная матрица размером 4 на 4. В таком случае мы можем раскрыть скобки и получим такое выражение:

$$  w' = P^{-1}(P^{T}X^{T}X)^{-1}P^{T}X^{T}y $$

**4**. Проведем аналогичное преобразование для скобки $(P^{T}X^{T}X)^{-1}$ = $(P^{T}(X^{T}X))^{-1}$ и раскроем ее:

$$  w' = P^{-1}(X^{T}X)^{-1}(P^{T})^{-1}P^{T}X^{T}y $$

**5**. Так как $ (P^{T})^{-1}P^{T} = E $, то формула обучения примет окончательный вид: 

$$  w' = P^{-1}(X^{T}X)^{-1}X^{T}y $$

**6**. Подставим $w'$ в формулу предсказания $a' = XPw'$:

$$a' = XPP^{-1}(X^{T}X)^{-1}X^{T}y$$

**7**. $PP^{-1} = E$, а значит: 

$$a' = X(X^{T}X)^{-1}X^{T}y$$

**Следовательно, $a = a'$, то есть результат предсказания не изменится. ЧТД.**

## 3. Алгоритм преобразования

**Алгоритм**

Нужно домножить матрицу признаков (с нулевым столбцом из единиц) на любую обратимую матрицу размером 4 на 4. В таком случае нам удастся защитить данные и при этом не испоортить качество модели линейной регресссии.

**Обоснование**

Доказательство правильности алгоритма преобразования приведено в шаге **2**.

## 4. Проверка алгоритма

In [5]:
# Домножим матрицу признаков на обратимую матрицу
matrix = np.array([
                 [5, 2, 8, 6],
                 [3, 12, 5, 14],
                 [9, 7, 2, 5],
                 [2, 4, 10, 6]
])
features_inv = data_features @ matrix
display(features_inv)

,0,1,2,3
0,446530.0,347698.0,99423.0,248586.0
1,342140.0,266556.0,76240.0,190650.0
2,189087.0,147348.0,42145.0,105406.0
3,375367.0,292160.0,83525.0,208806.0
4,234989.0,183038.0,52348.0,130898.0
...,...,...,...,...
4995,321388.0,250244.0,71560.0,178904.0
4996,471704.0,367212.0,104980.0,262482.0
4997,305164.0,237548.0,67920.0,169792.0
4998,294377.0,229178.0,65548.0,163832.0


In [6]:
# Проверка матрицы на обртимость
inverse = inv(matrix)
print(inverse)

[[ 0.10223426 -0.04062288  0.08530806 -0.07853758]
 [-0.29451591 -0.00880162  0.11848341  0.21631686]
 [-0.0521327  -0.0521327   0.00947867  0.16587678]
 [ 0.24915369  0.10629655 -0.12322275 -0.22782668]]


Матрица обратима.

In [7]:
# Создадим класс модели линейной регрессии
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

# Проверим модель на исходных данных
model = LinearRegression()
model.fit(data_features, data_target)
predictions = model.predict(data_features)
print("Показтель метрики r2_score при исходной матрице признаков:", round(r2_score(data_target, predictions), 5))

Показтель метрики r2_score при исходной матрице признаков: 0.42495


In [8]:
# Проверим модель на измененных данных
model = LinearRegression()
model.fit(features_inv, data_target)
predictions = model.predict(features_inv)
print("Показтель метрики r2_score при измененной матрице признаков:", round(r2_score(data_target, predictions), 5))

Показтель метрики r2_score при измененной матрице признаков: 0.42495


## 4. Общий вывод

**Вывод**:
Проведено сравнение моделей до и после домножения матрицы на обратимую матрицу. Качество модели осталось неизменным.